In [ ]:
import gym

In [ ]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space

In [ ]:
height, width, channels

In [ ]:
actions

In [ ]:
env.unwrapped.get_action_meanings()

#### Test Random Environment with OpenAI Gym

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print(f'Episode:{episode}, Score:{score}')
env.close()


#### Create a Deep Learning Model with Keras

In [ ]:
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Conv2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model


In [ ]:
model = build_model(height, width, channels, actions.n)
model.summary()

#### Build Agent with Keras-RL

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps',
                                  value_max=1, value_min=.1, value_test=.2,
                                  nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                   enable_dueling_network=True, dueling_type='avg',
                   nb_actions=actions, nb_steps_warmup=1000)
    return dqn

In [ ]:
# Train model
dqn = build_agent(model, actions.n)
dqn.compile(Adam(learning_rate=1e-4))
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

In [ ]:
# Test model
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

#### Saving and Reloading from Memory

In [ ]:
dqn.save_weights('spaceinvaders_10k_weights.h5f')

In [ ]:
del model, dqn

In [ ]:
model = build_model(height, width, channels, actions.n)
dqn = build_agent(model, actions.n)
dqn.compile(Adam(learning_rate=1e-4))
dqn.load_weights('spaceinvaders_10k_weights.h5f')